<span type="title">JDBC</span> | <span type="update">2018-10-04</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍使用 JDBC API 操纵数据库的流程、方法和演变过程。JDBC 的包位于 javax.sql 中，其包含了 Connection、DataSource、Statement、ResultRow等常见的数据库操纵类。本章亦介绍了 BeanUtils 和 DBUtils，前者用来将数据库数据映射为 Java Bean（POJO），后者用来线程安全的进行增删改查。</p></span>

# JDBC 基础概念



## 获取数据库连接 Connection 对象

**直接使用驱动获取连接**

JDBC 是 Java 程序连接数据库的一组通用 API 规范。通过 JDBC 访问数据库不需要代码耦合任何专有数据库厂商的驱动程序，比如 Mysql 或者 Oracle，以及 Microsoft SQL Server。通过指定驱动程序 Driver、JDBC访问地址（包含数据库协议、子协议以及地址、端口和数据库）、用户名和密码即可通过 `Driver.conncet` 获得 Connection 对象。通过此对象操纵数据库的表，进行增删改查的操作。

下面展示了一段测试代码：注意，用户名和密码常用 Properties 字典表示，作为附加信息传递到 Driver。注意 JDBC 地址包含jdbc协议和mysql子协议，在子协议后有冒号和双斜杠，然后是数据库地址、数据库名。如果是本地3306，可以省略 localhost:3306，直接三个斜杠，跟上数据库名称即可。

```java
Driver driver = new com.mysql.jdbc.Driver();
String url = "jdbc:mysql://localhost:3306/log";
//注意，如果是本地地址和本地端口，可以使用:jdbc:mysql:///log 简写
Properties info = new Properties();
info.put("user","corkine");
info.put("password","password");

Connection connection = driver.connect(url,info);
System.out.print(connection);
```

**使用外部属性文件通过反射获取连接**

一个稍微进化的版本是：从属性文件中获取用户名、密码等数据库配置信息，这里的 InputStream 使用了 FileInputStream 而不是 `InputStream in = new JDBCTools().getClass().getClassLoader().getResourceAsStream("jdbc.properties");` 的原因是，我们希望最后的程序打包成 jar，同时属性文件放在外部。注意这里的 Driver 是通过反射 `Class.forName("xxx").newInstance()` 来进行创建的。这种创建方式会转型为 Dirver 接口类。

```java
public static Connection getBasicConnection() throws Exception {
    InputStream in = new FileInputStream(System.getProperty("user.dir") +File.separator + "jdbc.properties");
    Properties properties = new Properties();
    properties.load(in); in.close();

    Properties info = new Properties();
    info.put("user",properties.getProperty("user"));
    info.put("password",properties.getProperty("password"));

    Driver driver = (Driver) Class.forName(properties.getProperty("driver")).newInstance();
    Connection connection = driver.connect(properties.getProperty("url"),info);
    return connection;
}
```

**使用DriverManager获取连接**

相比较使用 Driver 获取连接，使用 DriverManager 的好处显而易见：可以同时管理多个数据库驱动程序驱动的数据库。其使用方法很简单 `DriverManager.getConnection(url,user,password)` 即可。需要注意的是，在获取连接前需要实例化驱动：

```
For example, the following code fragment returns the runtime Class descriptor for the class named java.lang.Thread:
Class t = Class.forName("java.lang.Thread") //A call to forName("X") causes the class named X to be initialized.
```

DM 只需要初始化驱动即可，而不需要获取此引用。其工作原理如下：当使用 getConnection 的时候，会通过 `getConnection(var0, var3, Reflection.getCallerClass());` 重载，之后会找到这个 Class，然后委托其获得连接：`Connection var7 = var6.driver.connect(var0, var1);`

```java
public static Connection getManagerConnection() throws Exception {
    InputStream in = new FileInputStream(System.getProperty("user.dir") + File.separator + "jdbc.properties");
    Properties properties = new Properties();
    properties.load(in);

    Properties info = new Properties();
    info.put("user",properties.getProperty("user"));
    info.put("password",properties.getProperty("password"));

    Class.forName(properties.getProperty("driver"));
    return DriverManager.getConnection(
                properties.getProperty("url"),
                properties.getProperty("user"),
                properties.getProperty("password"));
}
```

## 使用 Statement 对象进行增删改查

一个最基础的增、删、改操作如下：

```java
Connection connection = null;
Statement statement = null;
try {
    connection = getManagerConnection();
    statement = connection.createStatement();
    statement.executeUpdate(sql);
} catch (Exception e) {
    e.printStackTrace();
} finally {
    if (statement != null) {
        try {
            statement.close();
        } catch (SQLException e) {
            e.printStackTrace();
        }
    }
    if (connection != null) {
        try {
            connection.close();
        } catch (SQLException e) {
            e.printStackTrace();
        }
    }
}
```

很显然，造成这样代码的原因是 Java 的异常机制，因为数据库在失败时必须关闭，否则影响其余人连接。但是关闭也会出错，因此需要继续的 try catch。在这里接触了一个新的对象： Statement，其代表一次 SQL 操作。通过 `con.createStatement` 获取基本的 Statement 对象，通过 `executeUpdate` 进行更新/增加/删除操作。注意，statement 有两个子类 `CallableStatement, PreparedStatement` 前者用于函数调用，后者用于预编译语法调用。

为了简化操作，我们把处理异常的关闭 Statement、Connection 的操作放在一个单独的类方法中：

```java
public static void release(ResultSet resultSet,Statement statement,Connection connection) {
    if (resultSet != null) { ... }
    if (statement != null) { ... }
    if (connection != null) { ... }
}
```

然后写一个基础的 execute 方法：

```java
public static void execute(String sql){
    Connection connection = null;
    Statement statement = null;
    try {
        connection = getManagerConnection();
        statement = connection.createStatement();
        statement.executeUpdate(sql);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        release(statement,connection);
    }
}
```

最后用这两个方法提供 delect、insert 方法：

```java
public static void delect(String sql) { execute(sql);}
public static void insert(String sql) { execute(sql);}
```

## 使用 ResultSet 对象管理返回值

至于查询的话，需要接触 ResultSet 对象。

```java
public static ResultSet query(String sql) {
    Connection connection = null;
    Statement statement = null;
    try {
        connection = getManagerConnection();
        statement = connection.createStatement();
        return statement.executeQuery(sql);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        release(statement,connection);
    }
    return null;
}
```

注意，这里使用的是 `executeQuery` 而不是 `executeUpdate` 方法。返回的是 `ResultSet` 对象。

下面是这个对象的使用方法：

```java
public static void printQuery(String sql) {
    Connection connection = null;
    Statement statement = null;
    ResultSet set = null;
    try {
        connection = getManagerConnection();
        statement = connection.createStatement();
        set =  statement.executeQuery(sql);
        while (set.next()) {
            int id = set.getInt(1);
            String name = set.getString(2);
            String email = set.getString(3);
            Date birth = set.getDate(4);
            System.out.printf("id: %s\nname: %s\nemail: %s\nbirth: %s\n\n",id,name,email,birth);
        }
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(set,statement,connection);
    }
}
```

注意到，这里的 ResultSet 方法，其可以通过 `next()` 进行指针的遍历（类似于迭代器）。每次迭代返回结果集的一行。对于每行，通过 getXXX 来获取对应结果，传入参数可以为列名，或者为列的编号，编号从1开始。至于类型，则是通过强转完成的。如果不匹配或者无法强转，则报错。

需要注意的是，ResultSet 也需要进行关闭，并且在 Statement、Connection 关闭后无法获取。因此这里 release 了三个对象，并且是按照顺序进行的。 Java 和 SQL 的类型对应如下：

```
 * boolean - bit
 * short - smallint
 * int - integer
 * long - bigint
 * string - char, varchar, long varchar
 * byte - array binary, var binary
 * sql.date - date
 * sql.time - time
 * sql.timestamp - timestamp
```

# JDBC 高级技术

## 使用 prepareStatement 安全查询

使用基本的 Statement 容易被 SQL 注入，安全性很差。而通过子类 prepareStatement 不是采用字符串拼接的方式，可以避免注入，并且当执行多条相同语句的时候，可以避免编译耗时，直接执行，速度更优。

pS 的 SQL 语句如下：`INSERT into customers (name, email, birth) values (?,?,?,?)` 其中问号表示占位符，表示需要填入的一个元素。

```java
public static void update(String sql, Object...args) {
    Connection connection = null;
    PreparedStatement statement = null;
    try {
        connection = getManagerConnection();
        statement = connection.prepareStatement(sql);
        for (int i = 0; i < args.length; i++) {
            //注意，setXXX 的索引是从1开始的
            statement.setObject(i+1,args[i]);
        }
        statement.executeUpdate();
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,statement,connection);
    }
}
```

以上是改进过的 update 方法，statment 通过 `con.prepareStatement(sql)` 获取，需要传入准备好的一条 sql 语句，然后通过 setXXX(i,obj) 来设置指定类型的数据到索引 i 处。注意， i 从1开始计，表示第 i 个问号（从左到右）。一般而言，我们通过可变数组 args 和 for 循环来 setObject 到 statement。当设置好占位符后，通过 executeUpdate/Query 进行更新。

## 开启事务管理

事务指的是一次元操作，在以上的例子中，我们每次执行一条 SQL 语句都需要连接一次数据库，并且提交保存，然后再进行连接、保存。这样很消耗资源，并且不能撤销。如果在两个SQL语句间隔发生错误，那么我们将无法在本地撤销第一条SQL语句，这就造成了严重的问题。因此，我们可以通过事务：一次执行多条SQL语句，来避免中断的问题，当发生错误，我们进行回滚，当操作成功，则将多条同时提交。

事务区别于批处理，批处理指的是在本地写多条语句，然后一次交给服务器。而事务指的是每一条都上传服务器，但是在完全成功之前，可以撤销事务内的所有SQL语句。

事务的写法大致和之前类似，区别在于，现在执行查询时，不关闭 con 对象，只关闭 rs 和 stat 对象。并且在每次查询的时候，都需要传入 con 引用。在执行查询前需要关闭 con 的自动提交，在完全结束后，需要对 con 进行总的提交。

```java
Connection connection = null;
        try {
            //事务必须设置不自动提交，并且在事务完成后进行commit.
            //注意，之前的update会在最后close connection，现在则不能关闭connection，但是可以关闭每次update的statement
            connection = JDBCTools.getManagerConnection();
            connection.setAutoCommit(false);
            connection.setTransactionIsolation(Connection.TRANSACTION_READ_COMMITTED);
            //事务常见隔离级别：
            /*读未提交（未提交的数据也可以读）
            * 读已提交（只能读取已提交的数据）
            * 可重复读（重复读取一个数据不变）
            * 串行化（拒绝第二个用户用表）*/
            //Oracle默认数据库为读已提交的数据（不能避免多行、多次读数据不同，但可以避免读到未提交数据），mysql默认为可重复读（不能避免多行，但避免了多次读取值不同）
            //一般在mysql中设置即可，不需要使用代码：
            /*
            show variables like '%isolation%';
            help isolation
            set [global|session] transaction isolation level xxxx;
            * */

            String sql_a = "update customers set name = 'fuck' where id = 521";
            JDBCTools.update(connection,sql_a);
            //System.out.println(2/0);
            String sql_b = "update customers set name = 'LiuJin' where id = 521";
            JDBCTools.update(connection,sql_b);

            connection.commit();
        } catch (Exception e) {
            e.printStackTrace();
        } finally {
            JDBCTools.release(null,null,connection);
        }
```

## 使用数据池

## 获取自增主键

## 大文件存储

# 基于 JDBC 的 DAO 方法

## 使用 反射 构建 Java 对象

## 使用 BeanUtils 构建 Java 对象

# 基于 JDBC 的方法封装

# 基于 JDBC 的 DAO 模板

# 基于 DBUtils 的 DAO 模板